In [1]:
from langchain.chat_models import init_chat_model


In [3]:
model = init_chat_model("gemini-2.0-flash",model_provider="google-genai")

In [4]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [5]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)

In [15]:
from langchain_community.document_loaders import TextLoader

document = TextLoader(
    file_path="./chat.txt",
    encoding="UTF-8"
)

docs = document.load()


In [17]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

"""
    Text Splitting
"""


text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(documents=docs)

all_splits

[Document(metadata={'source': './chat.txt'}, page_content='14/09/24, 11:21\u202fam - Messages and calls are end-to-end encrypted. Only people in this chat can read, listen to, or share them. Learn more.\n25/10/24, 9:04\u202fam - Sivasu: public class Main {\n    public static void main(String[] args) {\n        AirlineReservationSystem ars = new AirlineReservationSystem();\n        Scanner scanner = new Scanner(System.in);\n\n        // Adding some flights for demo\n        ars.addFlight("AI101", "New York", 100);\n        ars.addFlight("AI102", "London", 80);\n        ars.addFlight("AI103", "Dubai", 50);\n\n        System.out.print("Enter your name to register: ");\n        String userName = scanner.next();\n        ars.registerUser(userName);\n        User user = ars.getUser(userName); // Get the registered user'),
 Document(metadata={'source': './chat.txt'}, page_content='int choice;\n        do {\n            System.out.println("\\nAirline Reservation System");\n            System.o

In [ ]:
_ = vector_store.add_documents(documents=all_splits)

In [ ]:
from typing_extensions import TypedDict, List
from langchain_core.documents import Document
class State(TypedDict):
    question : str
    context : List[Document]
    answer : str
    

In [33]:
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state['question'])
    return {
        "context" : retrieved_docs
    }

from langchain import hub
prompt = hub.pull("rlm/rag-prompt")

def generate(state: State):
    docContents = "\n".join( doc.page_content for doc in state["context"] )
    messages = prompt.invoke({
        "question" : state['question'],
        "context" : docContents
    })
    response = model.invoke(messages)
    return {
        "answer" : response.content
    }

In [34]:
from langgraph.graph import StateGraph, START

graphBuilder = StateGraph(State).add_sequence([retrieve, generate])
graphBuilder.add_edge(START, "retrieve")
graph = graphBuilder.compile()

In [ ]:
response = graph.invoke({
    'question' : "What is the relationship between the two person"
})

response["answer"]

'Based on the context, Sai Sanjay Shanmugam G and Sivasu seem to have a close, friendly relationship, as indicated by their casual banter and use of terms like "bromance." They engage in playful and humorous conversations. The context suggests a strong bond between them.'